In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [9]:
import os
import time
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from data import load_dataset
from model import DDSSynth, construct_architecture
from ddsp.archis import construct_synth
from ddsp.loss import MSSTFTLoss
from utils.plot import plot_batch_detailed
from dotmap import DotMap

In [17]:
args = {'path': '/home/huanghao.blur/datasets',
        'dataset': 'medley',
        'nbworkers': 1,
        
        'data': 'mel',
        
        'sr': 44100,
        'f0_estimate': 'dio',
        'fft_scales': [64, 6],
        'smooth_kernel': 8,
        
        'n_partial': 50,
        'filter_size': 64,
        'block_size': 160,
        'kernel_size': 15,
        'sequence_size': 200,
        'synth_type': 'basic',
        
        'model': 'vae',
        'layers': 'gru',
        'loss': 'msstft',
        'strides': [2,4,4,5],
        'n_hidden': 512,
        'n_layers': 4,
        'channels': 128,
        'kernel': 15,
        'encoder_dims': 16,
        'latent_dims': 16,
        'warm_latent': 200,
        'beta_factor': 1,
        
        'flow': 'iaf',
        'flow_length': 8,
        
        'early_stop': 60,
        'train_type': 'random',
        'plot_interval': 100,
        'batch_size': 64,
        'epochs': 100,
        'lr': 2e-4
       }

args = DotMap(args)

In [12]:
train_loader, valid_loader, test_loader, args = load_dataset(args)

In [20]:
encoder, decoder = construct_architecture(args)
# Construct synthesizer
synth = construct_synth(args)
# Finally construct the full model (first only AE)
model = DDSSynth(encoder, decoder, synth, args)
# Optimizer model
optimizer = optim.Adam(model.parameters(), lr=args.lr)
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=True, threshold=1e-7)

loss = MSSTFTLoss(args.scales)

In [23]:
x, f0, loud, y = next(iter(train_loader))

In [34]:
x.shape

torch.Size([64, 1, 32000])